# Practical Rankle Examples

_If you have not done so, please review the primary [Rankle](Rankle.ipynb) notebook first._

In [1]:
(ns rankle.examples
  (:require [cemerick.pomegranate :as pg]))

(pg/add-classpath "./src")

(require '[com.semperos.rankle :as r])

## Numbers

### Arithmetic

In [2]:
(r/+ 1)

1

In [3]:
(r/+ 1 2 3)

6

In [4]:
(r/+ 1 [1 2 3])

(2 3 4)

In [5]:
(r/+ [1 2 3] 1)

(2 3 4)

In [6]:
(r/+ [1 2 3] [4 5 6])

(5 7 9)

In [7]:
(r/+ [[1 2 3] [4 5 6]]
     [[7 8 9] [10 11 12]])

((8 10 12) (14 16 18))

In [8]:
;; More succinct way to get larger multidimensional sequences of numbers:
(r/reshape [2 3] (range 6))

[[0 1 2] [3 4 5]]

In [9]:
(r/+ (r/reshape [2 3] (range 6))
     (r/reshape [2 3] (range 6)))

((0 2 4) (6 8 10))

In [10]:
;; We'll start printing these as tables for ease of review,
;; but don't forget that these are just nested Clojure vectors:
(r/print-aligned
 (r/+ (r/reshape [2 3] (range 6))
      (r/reshape [2 3] (range 6))))

  0  2  4
  6  8 10


In [11]:
;; As long as the common frame (i.e., the first axes of the shape of each argument)
;; is the same, we can operate on data of different shapes:
(r/print-aligned
 (r/+ (r/reshape [2 3] (range 6))
      (r/reshape [2 3 4] (range 24))))

  0  1  2  3
  5  6  7  8
 10 11 12 13

 15 16 17 18
 20 21 22 23
 25 26 27 28



In [12]:
;; All of these things work for other arithmetic operations, too.
(r/print-aligned
 (r/* 2 [2 4 6]))

4 8 12


In [13]:
(r/print-aligned
 (r/* [2 4 6] [8 10 12]))

16 40 72


In [14]:
(r/print-aligned
 (r/* (r/reshape [2 3] (range 6))
      (r/reshape [2 3 4] (range 24))))

  0  0  0  0
  4  5  6  7
 16 18 20 22

 36 39 42 45
 64 68 72 76
 100 105 110 115



In [15]:
(r/print-aligned
  (r/- (r/reshape [2 3] (range 6))
       (r/reshape [2 3] (map inc (range 6)))))

 -1 -1 -1
 -1 -1 -1


In [16]:
(r/print-aligned
  (r// (r/reshape [2 3] (range 6))
       (r/reshape [2 3] (map inc (range 6)))))

   0 1/2 2/3
 3/4 4/5 5/6


## Strings